### Synthesis and Preparation of Cleaned Electoral and Campaign Finance Data

In [30]:
from warnings import simplefilter
simplefilter(action='ignore')
import numpy as np
import pandas as pd
import sys
sys.path.append('../scripts')
from name_cleaner import clean_names
from rapidfuzz import fuzz

The two datasets imported below were obtained from the United States [Federal Election Comission](https://www.fec.gov/data/browse-data/?tab=bulk-data), were cleaned and prepared within the notebooks entitled [house_campaign_finance_data_2016_prep.ipynb](https://github.com/neophyte577/campaign-finance-house-2016/blob/main/data-prep/house_campaign_finance_data_2016_prep.ipynb) and [house_election_results_2016_prep.ipynb](https://github.com/neophyte577/campaign-finance-house-2016/blob/main/data-prep/house_election_results_2016_prep.ipynb) in the data_prep/ directory of this repository, and will be merged to yield the datasets we will ultimately use for our analysis. Unfortunately, the FEC's electoral and campaign finance data did not overlap enough to allow simultaneous comprehensive analysis of the interrelationships among campaign contributions, votes/vote share, and electoral victory for all candidates: Many candidates listed in the financial dataset either did not progress beyond the primary process or for other reasons were excluded from the electoral data; and many candidates (mostly those belonging to third parties) who were included in the electoral dataset and did participate in the general election were absent from the finance data. Attempts to follow up on these discrepant data points manually were met with confirmation from the FEC that they simply did not have financial data for these latter cadidates for the election cycle under consideration. 

In light of these limitations, two datasets have been produced via two different merge procedures: A balanced, albeit rather small dataset including only those cadidates for whom **both** complete electoral and campaign finance data were available was obtained by an inner merge on candidates' FEC IDs; and another, substantially larger but more imbalanced dataset was produced using a left merge of the finance data with the electoral data on FEC ID, and includes no electoral information other than a single categorical variable indicating electoral victory or loss. 

The second dataset will serve as the focus of the parametric analyses and algorithmic classifications which follow, while the first will be analyzed in a separate notebook. It seems reasonable to expect that both will scale and complement one another nicely upon augmentation with data from additional election cycles.

In [31]:
finance = pd.read_csv('../data/processed/house_campaign_finance_data_2016_processed.csv') 

finance[['fec_id','first_name','last_name']] = finance[['fec_id','first_name','last_name']].astype(str)

election = pd.read_csv('../data/processed/house_election_results_2016_processed.csv')

election[['fec_id','first_name','last_name']] = election[['fec_id','first_name','last_name']].astype(str)

#### Fuzzy Matching on Names

Initial attempts at merging the two datasets on FEC ID number were largely successful, but produced some discrepancies and omissions to be addressed below. In particular, numerous candidates occurring in both appear to have both mismatched names ID numbers between the two datasets, necessitating that they be explicitly identified and manually verified via a fuzzy match on name and a cross-reference on ID, following an initial filter excluding those candidates whose respective names returned a positive fuzzy match but who campaigned in different states.

In [32]:
def is_similar(row, threshold=80):
    first_sim = fuzz.ratio(row['first_name_e'].lower(), row['first_name_f'].lower()) >= threshold
    last_sim = fuzz.ratio(row['last_name_e'].lower(), row['last_name_f'].lower()) >= threshold
    return first_sim and last_sim

combined = election.merge(finance, how='cross', suffixes=('_e', '_f'))

combined['similar'] = combined.apply(is_similar, axis=1)

mismatched = combined[(combined['similar']) & (combined['fec_id_e'] != combined['fec_id_f'])]

id_discrepancies_first_last = mismatched[['fec_id_e', 'first_name_e', 'last_name_e', 'state_abbr_e', 'fec_id_f', 'first_name_f', 
                                          'last_name_f', 'state_abbr_f']][mismatched['state_abbr_e']==mismatched['state_abbr_f']]

id_discrepancies_first_last 

,fec_id_e,first_name_e,last_name_e,state_abbr_e,fec_id_f,first_name_f,last_name_f,state_abbr_f
423461,H6GA04152,victor,armendariz,GA,H0GA04106,victor,armendariz,GA
488379,H6IL05168,vince,kolber,IL,H6IL05150,vince,kolber,IL
556250,H6IN06149,barry,welsh,IN,H6IN06206,barry,welsh,IN
988469,H4NH02258,jim,lawrence,NH,H6NH02238,jim,lawrence,NH
1327725,H6PA05133,kerith,taylor,PA,H4PA05062,kerith,taylor,PA
1435408,H6TN05314,steven,reynolds,TN,H6TN04200,steven,reynolds,TN
1464913,H6TX01238,shirley,mckellar,TX,H2TX01062,shirley,mckellar,TX
1516610,H6TX14124,michael,cole,TX,H2TX36134,michael,cole,TX
1681758,H6WA03234,jaime,beutler,WA,H0WA03187,jaime,beutler,WA
1687659,H6WA05171,cathy,rodgers,WA,H4WA05077,cathy,rodgers,WA


In [33]:
from rapidfuzz import fuzz
import pandas as pd 

def is_similar(row, threshold=90):
    return fuzz.ratio(row['name_e'].lower(), row['name_f'].lower()) >= threshold

combined = election.merge(finance, how='cross', suffixes=('_e', '_f'))

combined['similar'] = combined.apply(is_similar, axis=1)

mismatched = combined[(combined['similar']) & (combined['fec_id_e'] != combined['fec_id_f'])]

id_discrepancies_name= mismatched[['fec_id_e', 'name_e', 'state_abbr_e', 'fec_id_f', 'name_f', 'state_abbr_f']][mismatched['state_abbr_e']==mismatched['state_abbr_f']]

id_discrepancies_name

,fec_id_e,name_e,state_abbr_e,fec_id_f,name_f,state_abbr_f
423461,H6GA04152,victor armendariz,GA,H0GA04106,victor armendariz,GA
488379,H6IL05168,vince kolber,IL,H6IL05150,vince kolber,IL
556250,H6IN06149,barry welsh,IN,H6IN06206,barry welsh,IN
988469,H4NH02258,jim lawrence,NH,H6NH02238,jim lawrence,NH
1327725,H6PA05133,kerith taylor,PA,H4PA05062,kerith taylor,PA
1435408,H6TN05314,steven reynolds,TN,H6TN04200,steven reynolds,TN
1464913,H6TX01238,shirley mckellar,TX,H2TX01062,shirley mckellar,TX
1516610,H6TX14124,michael cole,TX,H2TX36134,michael cole,TX
1681758,H6WA03234,jaime beutler,WA,H0WA03187,jaime beutler,WA
1687659,H6WA05171,cathy rodgers,WA,H4WA05077,cathy rodgers,WA


In [34]:
election.loc[election['fec_id'].isin(id_discrepancies_name['fec_id_e']),'fec_id'] = list(id_discrepancies_name['fec_id_f'])

election.loc[election['fec_id'].isin(id_discrepancies_first_last['fec_id_e']),'fec_id'] = [id for id in list(id_discrepancies_first_last['fec_id_f']) if 
                                                                                           id not in list(id_discrepancies_name['fec_id_f'])]

#### Dataset Including Campaign Finance and Complete Electoral Data

As mentioned above, the following dataset includes only those candidates who appeared on the 2016 general election ballot and for whom complete vote count and vote share data were available. It is less comprehensive (in that it captures only a small subset of electoral defeats), but is also richer, more balanced, and hopefully more informative in most respects than the larger dataset, whose observations contain only campaign finance data and a single dichotomous variable ('winner') encoding electoral victory or defeat. 

Description and analysis of this smaller dataset is carried out in the notebook entitled [house_campaign_finance_analysis_with_vote_data_2016.ipynb](https://github.com/neophyte577/campaign-finance-house-2016/blob/main/analysis/supplementary-analysis-smaller-dataset/house_campaign_finance_analysis_with_votes_2016.ipynb) in the analysis/ directory of this repository.

The dataset is introduced here for later use in filtering non-competing incumbents from the larger dataset, the raw data for which included incumbents who did not seek reelection to their respective House seats and were therefore not of interest to the analysis which follows despite having reported nonzero contributions to the FEC during the 2015-2016 cycle.

In [35]:
house_with_votes = pd.merge(election, finance.drop(['first_name','last_name','name','state_abbr','party'], axis='columns'), on=['fec_id'], how='inner')

house_with_votes = house_with_votes[house_with_votes['votes']!='Unopposed']

display(house_with_votes.head(), house_with_votes.info())

<class 'pandas.core.frame.DataFrame'>
Index: 772 entries, 1 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_abbr  772 non-null    object 
 1   state       772 non-null    object 
 2   district    772 non-null    int64  
 3   fec_id      772 non-null    object 
 4   first_name  772 non-null    object 
 5   last_name   772 non-null    object 
 6   name        772 non-null    object 
 7   party       772 non-null    object 
 8   votes       772 non-null    object 
 9   vote_share  772 non-null    float64
 10  winner      772 non-null    int64  
 11  inc         770 non-null    object 
 12  tot_rec     772 non-null    float64
dtypes: float64(2), int64(2), object(9)
memory usage: 84.4+ KB


,state_abbr,state,district,fec_id,first_name,last_name,name,party,votes,vote_share,winner,inc,tot_rec
1,AL,Alabama,2,H0AL02087,martha,roby,martha roby,Republican,134886,0.487685,1,Incumbent,1404260.12
2,AL,Alabama,2,H6AL02167,nathan,mathis,nathan mathis,Democrat,112089,0.405262,0,Challenger,36844.00
3,AL,Alabama,3,H2AL03032,mike,rogers,mike rogers,Republican,192164,0.669318,1,Incumbent,1139022.37
4,AL,Alabama,3,H4AL03061,jesse,smith,jesse smith,Democrat,94549,0.329320,0,Challenger,9810.00
6,AL,Alabama,5,H0AL05163,mo,brooks,mo brooks,Republican,205647,0.666979,1,Incumbent,476225.18


None

In [36]:
house_with_votes[['district','votes']] = house_with_votes[['district','votes']].apply(pd.to_numeric, errors='coerce', axis='columns')

print(house_with_votes.dtypes)

state_abbr     object
state          object
district        int64
fec_id         object
first_name     object
last_name      object
name           object
party          object
votes           int64
vote_share    float64
winner          int64
inc            object
tot_rec       float64
dtype: object


In [37]:
display(house_with_votes.isna().sum())

display(house_with_votes[house_with_votes['inc'].isna()])

house_with_votes.loc[house_with_votes['name']=='steve lindbeck','inc'] = 'Challenger'
house_with_votes.loc[house_with_votes['first_name']=='vaitinasa','inc'] = 'Challenger'

state_abbr    0
state         0
district      0
fec_id        0
first_name    0
last_name     0
name          0
party         0
votes         0
vote_share    0
winner        0
inc           2
tot_rec       0
dtype: int64

,state_abbr,state,district,fec_id,first_name,last_name,name,party,votes,vote_share,winner,inc,tot_rec
12,AK,Alaska,0,H6AK00235,steve,lindbeck,steve lindbeck,Democrat,111019,0.360220,0,NaN,1102309.77
14,AS,American Samoa,0,H6AS00031,vaitinasa,hunkinfinau,vaitinasa hunkinfinau,Democrat,1581,0.133598,0,NaN,41670.00


In [38]:
print('Incumbency counts:\n', house_with_votes['inc'].value_counts().reindex(['Incumbent', 'Challenger', 'Open'], fill_value=0).rename_axis('') )
print()
print('Winners:', np.shape(house_with_votes[house_with_votes['winner']==1])[0])
print('Losers:',  np.shape(house_with_votes[house_with_votes['winner']==0])[0])

Incumbency counts:
 
Incumbent     365
Challenger    312
Open           95
Name: count, dtype: int64

Winners: 412
Losers: 360


#### Larger Dataset Excluding Vote Count and Vote Share

In [39]:
house = pd.merge(finance, election[['fec_id','winner']], on='fec_id', how='left')

unopposed_ids = election[election['votes']=='Unopposed']['fec_id']

house = house[~house['fec_id'].isin(unopposed_ids)]

house['winner'] = house['winner'].fillna(0)

display(house.head(), house.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1446 entries, 0 to 1473
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fec_id      1446 non-null   object 
 1   state_abbr  1446 non-null   object 
 2   first_name  1446 non-null   object 
 3   last_name   1446 non-null   object 
 4   name        1446 non-null   object 
 5   party       1446 non-null   object 
 6   inc         1444 non-null   object 
 7   tot_rec     1446 non-null   float64
 8   winner      1446 non-null   float64
dtypes: float64(2), object(7)
memory usage: 113.0+ KB


,fec_id,state_abbr,first_name,last_name,name,party,inc,tot_rec,winner
0,H6AK00045,AK,donald,young,donald young,Republican,Incumbent,1103561.86,1.0
1,H6AK00235,AK,steve,lindbeck,steve lindbeck,Democrat,NaN,1102309.77,0.0
3,H6AL01060,AL,larry,young,larry young,Republican,Challenger,178766.88,0.0
4,H6AL02167,AL,nathan,mathis,nathan mathis,Democrat,Challenger,36844.00,0.0
5,H0AL02087,AL,martha,roby,martha roby,Republican,Incumbent,1404260.12,1.0


None

In [40]:
display(house[house['inc'].isna()])

house.loc[house['name']=='steve lindbeck','inc'] = 'Challenger'
house.loc[house['first_name']=='vaitinasa','inc'] = 'Challenger'

display(house[house['inc'].isna()])

,fec_id,state_abbr,first_name,last_name,name,party,inc,tot_rec,winner
1,H6AK00235,AK,steve,lindbeck,steve lindbeck,Democrat,NaN,1102309.77,0.0
26,H6AS00031,AS,vaitinasa,hunkinfinau,vaitinasa hunkinfinau,Democrat,NaN,41670.00,0.0


,fec_id,state_abbr,first_name,last_name,name,party,inc,tot_rec,winner


In [41]:
print('Incumbency counts:\n', house['inc'].value_counts().reindex(['Incumbent', 'Challenger', 'Open'], fill_value=0).rename_axis('') )
print('Winners:', np.shape(house[house['winner']==1])[0])
print('Losers:',  np.shape(house[house['winner']==0])[0])

Incumbency counts:
 
Incumbent     414
Challenger    715
Open          317
Name: count, dtype: int64
Winners: 412
Losers: 1034


In [42]:
print(house_with_votes.dtypes)

house_with_votes[['district','votes']] = house_with_votes[['district','votes']].apply(pd.to_numeric, errors='coerce', axis='columns')

state_abbr     object
state          object
district        int64
fec_id         object
first_name     object
last_name      object
name           object
party          object
votes           int64
vote_share    float64
winner          int64
inc            object
tot_rec       float64
dtype: object


In [43]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 
          'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 
          'WA', 'WV', 'WI', 'WY']

house_with_votes = house_with_votes[house_with_votes['state_abbr'].isin(states)]

In [44]:
house_with_votes.to_csv('../data/analysis-ready/house-2016-with-votes-smaller-dataset.csv', index=False)

#### Removing Non-Competing Incumbents

As can be seen from the output of the cells below, the is a large discrepancy in incumbent count between the vote-inclusive dataset and the larger dataset is attributable to the 50 representatives who had raised campaign funds during the 2015-2016 cycle (and were therefore included in the finance data), but were either retiring from public office or were seeking a different office (and were therefore excluded from the electoral data). As this project is meant to investigate the relationship between campaign contributions and election outcomes, these candidates were excluded from the larger dataset.

In [ ]:
retiree_df = pd.read_csv('../data/raw/retiring_incumbents_house_2016.csv')

retiree_df = clean_names(retiree_df)

retirees = retiree_df['name']

len(retirees)

47

In [21]:
display(np.shape(house[house['inc']=='Incumbent'])[0], np.shape(house_with_votes[house_with_votes['inc']=='Incumbent'])[0])

discrepant_inc_ids = list(set(house[house['inc']=='Incumbent']['fec_id']) - set(house_with_votes[house_with_votes['inc']=='Incumbent']['fec_id']))

print('Number of discrepant incumbents in finance dataset:', len(discrepant_inc_ids))

discrepant_incumbents = house[house['fec_id'].isin(discrepant_inc_ids)]['name']

414

362

Number of discrepant incumbents in finance dataset: 52


In [22]:
from rapidfuzz import fuzz
import pandas as pd

def is_similar(name1, name2, threshold=80):
    return fuzz.ratio(name1.lower(), name2.lower()) >= threshold

# Create all combinations of names between discrepant_incumbents and name_retirees
combined = pd.DataFrame({
    'name_discrepant_incumbents': discrepant_incumbents.repeat(len(retirees)).reset_index(drop=True),
    'name_retirees': pd.concat([retirees] * len(discrepant_incumbents)).reset_index(drop=True)
})

combined['similar'] = combined.apply(
    lambda row: is_similar(row['name_discrepant_incumbents'], row['name_retirees']), axis=1
)

matches = combined[combined['similar']]

In [23]:
display(np.shape(matches)[0], matches) 

house = house[~house['name'].isin(matches['name_discrepant_incumbents'])] # Drop all fuzzy matches between retiree data and names of discrepant incumbents

df1 = retiree_df[~retiree_df['name'].isin(matches['name_retirees'])]['name'].reset_index(drop=True).sort_values()

df2 = house[house['fec_id'].isin(discrepant_inc_ids)][~house['name'].isin(matches['name_discrepant_incumbents'])]['name'].reset_index(drop=True).sort_values()

display(pd.DataFrame({'In Retiree List, Not in Finance':df1, 'In Finance, Not In Retiree List':df2}))

# View names from retiree data which did not fuzzy match any names among discrepant incumbents

41

,name_discrepant_incumbents,name_retirees,similar
74,ann kirkpatrick,ann kirkpatrick,True
108,matt salmon,matt salmon,True
163,sam farr,sam farr,True
200,lois capps,lois capps,True
274,janice hahn,janice hahn,True
315,loretta sanchez,loretta sanchez,True
414,john carney,john carney,True
431,jefferson miller,jeff miller,True
477,gwen graham,gwen graham,True
517,ander crenshaw,ander crenshaw,True


,"In Retiree List, Not in Finance","In Finance, Not In Retiree List"
0,scott rigell,aumua amata
1,alan nunnelee,eleanor norton
2,ed whitfield,corrine brown
3,john boehner,madeleine bordallo
4,mark takai,kyle takai
5,michael grimm,patrick nunnelee
6,NaN,renee ellmers
7,NaN,george kelly
8,NaN,pedro pierluisi
9,NaN,edward rigell


Of the above-listed candidates who remain after identifying the majority of the errant incumbents as retirees, none achieved reelection apart from George Joseph "Mike" Kelly.

In [24]:
house.loc[house['name']=='george kelly', 'winner'] = 1

house.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1402 entries, 0 to 1473
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fec_id      1402 non-null   object 
 1   state_abbr  1402 non-null   object 
 2   first_name  1402 non-null   object 
 3   last_name   1402 non-null   object 
 4   name        1402 non-null   object 
 5   party       1402 non-null   object 
 6   inc         1402 non-null   object 
 7   tot_rec     1402 non-null   float64
 8   winner      1402 non-null   float64
dtypes: float64(2), object(7)
memory usage: 141.8+ KB


In [25]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 
          'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 
          'WA', 'WV', 'WI', 'WY']

house = house[house['state_abbr'].isin(states)]

In [26]:
house.to_csv('../data/analysis-ready/house-2016-larger-dataset.csv', index=False)

### Data Dictionary for Cleaned and Prepped Dataset

The final dataset produced above the contains nine columns corresponding to five variables which it is presumed are self-explanatory (viz., 'fec_id', 'state_abbr', 'first_name', 'last_name', and 'name', all strings, none of which are used in the displays and analyses which follow, but were preserved for data integrity and diagnostic purposes) and four variables of interest to the remaining sections of this notebook:

1) Total Campaign Receipts ('tot_rec') - float64: Denoted in the [Analysis](#Parametric-Analysis) section of this notebook as $C$, and alternately referred to synonymously as "total campaign contributions," "contributions," "receipts," and similar variations throughout the remainder of the notebook. A decision was made to incorporate this single variable to the exclusion of all other currency-valued columns in the finance data after a cursory check of the finance data revealed that 'tot_rec' approximately equaled the sum of the majority of the other currency-valued columns, and clearly indicates the total amount of money received by a campaign from all sources for which the FEC mandates reporting. In the interest of parsimony and to prevent overfitting, these redundant columms were omitted from the data for the purposes of the present analysis, although this project leaves ample room to incorporate additional currency-valued features in the generalization to a larger dataset including additional election cycles, to drill down further and explore, e.g., whether campaigns which took out larger loans (n.b. [loans also constitute campaign contributions](https://www.fec.gov/help-candidates-and-committees/candidate-taking-receipts/types-contributions/) according to FEC rules) stood a greater or lesser chance of victory than those who relied more heavily on transfers from committees. However, for the purposes of investigating the more straightforward hypothesis regarding the extent to which campaign contributions drive electoral outcomes, we will postpone consideration of such nuances and focus exclusively on total campaign receipts as a measure of the amount of money infused into a campaign.

2) Incumbency ('inc') - string: Denoted in the [Analysis](#Parametric-Analysis) section as $I$, this variable encodes the three possible competitive dispositions of the 2016 House candidates, namely 'Incumbent', 'Challenger', and 'Open' seat contender: 
    - 'Incumbent': An **incumbent** is a political candidate who currently holds the position for which they are running.
    - 'Challenger': As the name suggests, a **challenger** is a political candidate who is running against an incumbent and other challengers.
    - 'Open': If an elected office/position is for any reason vacated (e.g., if the previous holder of that office retires or resigns), then the race is considered **open**, and any candidate running for an open seat in the House will hereafter be referred to as an 'open seat contender' or 'open contender.'

    Since it was surmised at the outset that incumbency is a major driver of electoral outcome which may consitute a significant covariate with total campaign contributions (and therefore a potential confounding factor if omitted), it was deemed necessary to include a variable indicating whether a candidate was incumbent or not. Helpfully, the FEC finance data went one step further and also delineated non-incumbents by whether they were challenging an incumbent for their seat or vying for an open seat. As will be seen below, these differences in disposition prove crucial to understanding the role of campaign contributions in determining election outcomes.

3) Political Party Affiliation ('party') - string: Due to the paucity of third party candidates for which the FEC received data for the 2015-2016 House election cycle, candidates were classified according to party affiliation as 'Independent', 'Republican', 'Democrat', or 'Other'. Should subsequent data from other election cycles contain more complete financial records for third party campaigns, it may be of interest to explicitly incorporate, e.g., the Green Party, Libertarian Party, etc. into the analysis.

4) Electoral Outcome ('winner') - string: Denoted in the [Analysis](#Parametric-Analysis) section as $W$, this is a dichotomous string column encoding electoral victory or loss.
